# Basic usage of `DEMAP`

In this notebook, we will see how to install and use `DEMAP` to extract river
network from DEM.

## Install `DEMAP`

Download the source code from Github and install it using `pip` locally:

```Shell
git clone https://github.com/laijingtao/demap.git
pip install ./demap
```

In [1]:
import demap

## Accelerate with `Numba`

Many methods in `DEMAP` can be accelerated using `Numba`. If `Numba` is already
installed, these methods will automatically detect it and run much faster.
[Here](https://numba.pydata.org/numba-doc/latest/user/installing.html) is the
guide for installing `Numba`.

## Load DEM

In [2]:
dem_ds = demap.load_dem('olympics_500m_dem.tif')

`dem_ds` is a `xr.Dataset` contains the DEM and relevant geospatial information.

`Demap` is built around `Xarray`, and most methods can be used through demap's xarray [accessor](https://docs.xarray.dev/en/latest/internals/extending-xarray.html).

For example, the following `process_dem` method will fill the local depressions in DEM, calculate flow direction and drainage area. These informaiton will be stored in `dem_ds`.

In [ ]:
dem_ds.demap.process_dem()

`dem_ds` can be saved as a netcdf file for future use:

In [4]:
dem_ds.to_netcdf('olympics_500m_dem_processed.nc')

Saved results can be loaded again (note `decode_coords="all"` is necessary to load the geospatial information):

In [5]:
import xarray as xr

dem_ds = xr.load_dataset('olympics_500m_dem_processed.nc', decode_coords="all")

## Build a stream network

Given a drainage area threshold for stream head, a network of streams can be built from the flow direction information stored in `dem_ds`:

In [6]:
stream_ds = dem_ds.demap.build_stream_network(drainage_area_threshold=1e6)

We can save the stream network in shapefile to view it in GIS software:

In [10]:
demap.network_to_shp(stream_ds, 'olympics_500m_stream.shp')

## Extract a catchment

Given an outlet, we can extract the stream network in a catchment.

In [7]:
outlet_x, outlet_y = 403689, 5266903
example_network = stream_ds.demap.extract_from_xy(outlet_x, outlet_y, direction='up')

Split this network into a list of streams:

In [8]:
example_stream_list = example_network.demap.split_stream_network()

and then plot the longitudinal profiles of all streams:

In [ ]:
import matplotlib.pyplot as plt

for s in example_stream_list:
    plt.plot(s['distance_upstream'], s.demap.get_value(dem_ds['dem']))

`get_value()` can extract the relevant values along the stream from some given data. Here, we use it to extract elevation along the stream.

### Visualize the stream network

`Demap` provides some quick visualization methods:

In [ ]:
plt.figure(figsize=(10, 10))
ax = plt.gca()

dem_ds.demap.plot_hillshade(ax, clip_ref=example_network, clip_padding=2e3)
example_network.demap.plot_stream(ax)